In [4]:
import pandas as pd
from pathlib import Path

RAW_DIR = Path("eaglei_outages")   # 你下载解压的目录
OUT_DIR = Path("data")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ---- 1) 读入并立即过滤 Puerto Rico 数据以节省内存 ----
files = sorted(RAW_DIR.glob("eaglei_outages_*.csv"))  # 只读取outage文件，不包括coverage_history

usecols = ["fips_code", "run_start_time", "sum"]  # 实际的列名
dfs = []
for fp in files:
    print(f"读取 {fp.name}...")
    df_temp = pd.read_csv(fp, usecols=usecols, dtype={"fips_code": "string"})
    # 立即过滤只保留 Puerto Rico (FIPS = 72xxx)
    df_temp = df_temp[df_temp["fips_code"].str.startswith("72", na=False)]
    if len(df_temp) > 0:
        dfs.append(df_temp)
    print(f"  保留 {len(df_temp)} 行 PR 数据")

df = pd.concat(dfs, ignore_index=True)
print(f"\n总共 {len(df)} 行 Puerto Rico 数据")

# 重命名列以便后续使用
df = df.rename(columns={
    "fips_code": "fips",
    "run_start_time": "timestamp",
    "sum": "customers_out"
})

# ---- 2) 时间处理 + 指标 ----
df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)
df["month"] = df["timestamp"].dt.to_period("M").astype(str)

# 注意：由于没有 customers_total，无法计算 out_pct
# 我们只能基于绝对停电数量进行分析

# 每条记录代表 15 分钟，将停电客户数转换为停电小时数（客户*小时）
df["outage_customer_hours"] = df["customers_out"] * 0.25

# ---- 3) 事件次数：检测停电事件开始 ----
# 当停电客户数超过一定阈值时算作事件开始
TH = 1000  # 1000个客户停电算"事件开始"，你可以调整这个值
df = df.sort_values(["fips", "timestamp"])
df["is_out"] = df["customers_out"] > TH
df["is_start"] = df["is_out"] & (~df.groupby("fips")["is_out"].shift(1).fillna(False))

# ---- 4) 月度聚合 ----
monthly = (
    df.groupby(["fips", "month"], as_index=False)
      .agg(
          total_outage_customer_hours=("outage_customer_hours", "sum"),
          event_count=("is_start", "sum"),
          peak_customers_out=("customers_out", "max"),
          avg_customers_out=("customers_out", "mean"),
      )
)

# （可选）做一个 Maria 前后窗口
# monthly = monthly[(monthly["month"] >= "2017-01") & (monthly["month"] <= "2018-12")]

monthly.to_json(OUT_DIR / "pr_outage_monthly.json", orient="records")


读取 eaglei_outages_2014.csv...
  保留 0 行 PR 数据
读取 eaglei_outages_2015.csv...
  保留 0 行 PR 数据
读取 eaglei_outages_2016.csv...
  保留 0 行 PR 数据
读取 eaglei_outages_2017.csv...
  保留 0 行 PR 数据
读取 eaglei_outages_2018.csv...
  保留 0 行 PR 数据
读取 eaglei_outages_2019.csv...
  保留 0 行 PR 数据
读取 eaglei_outages_2020.csv...
  保留 0 行 PR 数据
读取 eaglei_outages_2021.csv...
  保留 219002 行 PR 数据
读取 eaglei_outages_2022.csv...
  保留 205446 行 PR 数据

总共 424448 行 Puerto Rico 数据


C:\Users\Arthur\AppData\Local\Temp\ipykernel_65296\2034968375.py:34: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df["month"] = df["timestamp"].dt.to_period("M").astype(str)
C:\Users\Arthur\AppData\Local\Temp\ipykernel_65296\2034968375.py:47: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["is_start"] = df["is_out"] & (~df.groupby("fips")["is_out"].shift(1).fillna(False))
